# Predição: Análise local de extremos
## Universidade Federal de Alagoas - UFAL
## Centro de Tecnologia - CTEC
## Programa de Pós-Graduação Recursos Hídricos e Saneamento - PPGRHS
Disciplina de Estatística
Clebson Farias

In [1]:
import pandas as pd
import numpy as np
import plotly as py

from files.ons import Ons
from files.ana import Ana
from series.flow import Flow
from series.chuva import Chuva

In [2]:
file = "dados_com_evap_manso.csv"
dados = pd.read_csv(file, index_col=0, parse_dates=True)
#Convetendo obj pandas para Series Flow
dados = Flow(pd.DataFrame(dados["MANSO"]))
fig, data = dados.plot_hydrogram()
py.offline.plot(fig, filename='gráficos/histo.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/histo.html'

# Máximas Anuais

In [3]:
peaks_max = dados.maximum(station="MANSO")
peaks_max.peaks.head()

,peaks
1982-02-03,1126.0
1983-01-19,1193.0
1983-12-02,1118.0
1985-01-08,1157.0
1986-02-21,1088.0


In [4]:
fig, data = peaks_max.plot_hydrogram()
py.offline.plot(fig, filename='gráficos/histo_max.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/histo_max.html'

## Máxima Verossimilhança

In [5]:
peaks_max.mvs()
peaks_max.fit

(-5.721987476983145, 149.63189576303387, 2.1852412410130952)

In [6]:
estimador = 'mvs'
title = 'Manso Nat'
type_function = 'cumulative'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_anual_mvs.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/distrib_anual_mvs.html'

In [7]:
estimador = 'mvs'
title = 'Manso Nat'
type_function = 'density'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_D_anual_mvs.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/distrib_D_anual_mvs.html'

In [8]:
period_return = 5
mag = peaks_max.magnitude(period_return, estimador)
mag

150.013760014353

In [9]:
period_return = 10
mag = peaks_max.magnitude(period_return, estimador)
mag

150.01379752442776

# Momentos L

In [10]:
peaks_max.mml()
peaks_max.fit

[0.20638553042392516, 1003.1174140035938, 333.9667588766301]

In [11]:
estimador = 'mml'
title = 'Manso Nat'
type_function = 'cumulative'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_anual_mml.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/distrib_anual_mml.html'

In [12]:
estimador = 'mml'
title = 'Manso Nat'
type_function = 'density'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_d_anual_mml.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/distrib_d_anual_mml.html'

In [13]:
period_return = 15
mag = peaks_max.magnitude(period_return, estimador)
mag

2214.7249828256026

In [14]:
period_return = 5
mag = peaks_max.magnitude(period_return, estimador)
mag

1640.64771862514

In [15]:
period_return = 2
mag = peaks_max.magnitude(period_return, estimador)
mag

1251.9819896874033

# Séries de Durações Parciais

In [16]:
station = "MANSO"
#'stationary' or 'events_by_year'
type_threshold = 'stationary' 
#'flood' or 'drought'
type_event = 'flood'
#'media', 'mediana' or 'autocorrelation' 
type_criterion = 'median'
#Peaks or percentil
value_threshold = 0.75

peaks_sdp = dados.parcial(station, type_threshold, type_event, type_criterion, value_threshold)
peaks_sdp.peaks.head()

,Duration,Start,End,peaks
1982-02-03,114,1982-01-01,1982-04-25,1126.0
1983-01-19,120,1982-12-15,1983-04-14,1193.0
1983-12-02,195,1983-11-02,1984-05-15,1118.0
1984-11-23,4,1984-11-21,1984-11-25,418.0
1985-01-08,130,1984-11-29,1985-04-08,1157.0


In [17]:
fig, data = peaks_sdp.plot_hydrogram(title='SDP-Manso')
py.offline.plot(fig, filename='gráficos/histo_sdp.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/histo_sdp.html'

## Máxima Verossimilhança

In [18]:
peaks_sdp.mvs()
peaks_sdp.fit

/home/clebsonpy/Documentos/Projetos/HydroComp/venv/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:2062: RuntimeWarning:

divide by zero encountered in true_divide

/home/clebsonpy/Documentos/Projetos/HydroComp/venv/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1037: RuntimeWarning:

invalid value encountered in subtract

/home/clebsonpy/Documentos/Projetos/HydroComp/venv/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2306: RuntimeWarning:

invalid value encountered in double_scalars



(-0.2160562934977424, 217.99585474281037, 638.0504071922765)

In [19]:
title = 'Manso Nat'
type_function = 'cumulative'
fig, data = peaks_sdp.plot_distribution(title, type_function)
py.offline.plot(fig, filename='gráficos/distrib_sdp_mvs.html')

'file:///home/clebsonpy/Documentos/Projetos/HydroComp/gráficos/distrib_sdp_mvs.html'

In [21]:
period_return = 5
mag = peaks_sdp.magnitude(period_return)
mag

1085.3703015123128

In [24]:
period_return = 10
mag = peaks_sdp.magnitude(period_return)
mag

1375.4715763026747

In [26]:
period_return = 15
mag = peaks_sdp.magnitude(period_return)
mag

1526.086579216705